[View in Colaboratory](https://colab.research.google.com/github/ahracho/TIL/blob/master/Fluent_Python/5_First_Class_Function.ipynb)

# Part III.  객체로서의 함수
## 5. 일급함수

파이썬의 함수는 일급 객체이다. 일급 객체의 요건는 다음과 같다.  
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

함수를 객체로 처리하는 실용적인 방법과 영향에 대해 집중적으로 살펴보자.

### 5.1. 함수를 객체처럼 다루기

In [1]:
def factorial(n) :
  '''return n!'''
  return 1 if n < 2 else n * factorial(n-1)

print(factorial(42))
print(factorial.__doc__) # 도움말  텍스트 생성
print(type(factorial))

1405006117752879898543142606244511569936384000000000
return n!
<class 'function'>


In [2]:
fact = factorial
print(fact)

print(fact(5))
print(map(factorial, range(11)))
print(list(map(fact, range(11)))) # map의 결과를 list로

<function factorial at 0x7f04d06a27b8>
120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


### 5.2. 고위 함수

함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 고위 함수(High-Ordered function)라고 한다.

sorted() 내장 함수도 일급 함수의 예이다. key 옵션에 선택적으로 함수를 전달받아 정렬할 항목에 적용가능하다.

In [3]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len) # len 함수를 인수로 전달

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [4]:
def reverse(word):
  return word[::-1]

print(reverse)
sorted(fruits, key=reverse) # 각 단어 철자를 거꾸로 한 결과를 기준으로 정렬 - 기존 리스트에는 변화 X

<function reverse at 0x7f04d06a2b70>


['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

#### 5.2.1. map(), filter(), reduce()의 대안

파이썬 3에 map(), filter() 함수는 여전히 내장함수이지만, 지능형 리스트나 제너레이터의 등장으로 활용도가 떨어졌다.

In [6]:
print(list(map(fact, range(6))))
print([fact(n) for n in range(6)]) # 의미 전달 / 가독성 훨씬 좋음
print(list(map(factorial, filter(lambda n : n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2]) # 의미 전달 / 가독성 훨씬 좋음

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


In [7]:
from functools import reduce # 파이썬 3에선 reduce가 내장함수가 아니다
from operator import add
print(reduce(add, range(100))) # reduce는 연속된 항목에 연산을 적용해서 하나의 값으로 줄이는 역할
print(sum(range(100))) # sum이 훨씬 사용하기 쉬움

4950
4950


### 5.3. 익명함수

lambda 키워드는 익명함수를 생성한다. 람다 본체에는 할당문이나 while, try 등 파이썬 문장을 사용할 수 없다. 

익명함수는 인수 목록 안에서 유용하게 사용된다. 고위 함수의 인수로 사용하는 방법 외에 익명 함수는 파이썬에서는 거의 사용되지 않는다. 

In [8]:
sorted(fruits, key=lambda word:word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### 5.4. 일곱 가지 맛의 콜러블 객체

호출 연산자 ()를 사용할 수 있는 콜러블 객체의 종류

1. 사용자 정의 함수: def나 람다 함수
2. 내장 함수
3. 내장 메서드
4. 메서드 : 클래스 본체에 정의된 함수
5. 클래스 : 생성자
6. 클래스 객체 : 클래스가 \_\_call\_\_()를 구현하면, 이 클래스의 객체는 함수로 호출될 수 있다.
7. 제너레이터 함수

### 5.4. 사용자 정의 콜러블형

파이썬 함수가 실제 객체일 뿐만 아니라, 모든 파이썬 객체가 함수처럼 동작하게 만들 수 있다. \_\_call\_\_() 메서드를 구현하면 된다.

In [9]:
import random

class BingoCage:
  def __init__(self, items):
    self._items = list(items)
    random.shuffle(self._items)
   
  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError("pick from empty BingoCage")
      
  def __call__(self):
    return self.pick()
  
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo()) # pick이 호출됨
print(callable(bingo))

0
1
True


### 5.6. 함수 인트로스펙션 (Introspection)

함수를 객체로 다루는 것과 관련된 속성을 알아보자. 일반적인 사용자 정의 클래스와 마찬가지로 함수는 \_\_dict\_\_ 속성을 이용해서 객체에 할당된 사용자 속성을 보관한다. 

In [10]:
dir(factorial)
# 결과값은 일반적으로 파이썬 객체에 존재하는 속성들

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [0]:
# Django 예제
def upper_case_name(obj):
  return ("%s %s" % (obj.first_name, obj.last_name)).upper()

upper_case_name.short_description = "Customer Name"

In [11]:
class C : pass
obj = C()
def func() : pass
sorted(set(dir(func)) - set(dir(obj))) # 일반 객체에는 없고 함수에만 있는 속성
'''
['__annotations__', # 매개변수와 반환값에 대한 주석
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__', # 디폴트 매개변수 값
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__'] # full name?
'''

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

### 5.7. 위치 매개변수에서 키워드 전용 매개변수까지



In [12]:
# * 뒤에 위치한 매개변수는 모두 키워드 전용 인수; *가 남은 위치인수를 흡수
def tag(name, *content, cls=None, **attrs): # cls 매개변수는 키워드 인수로만 전달 가능
  if cls is not None:
    attrs['class'] = cls
  if attrs:
    attr_str = ''.join(' %s=%s'%(attr, value) for attr, value in sorted(attrs.items()))
  else :
    attr_str = ''
    
  if content:
    return '\n'.join('<%s%s>%s<\%s>' % (name, attr_str, c, name) for c in content)
  else :
    return '<%s%s />' % (name, attr_str)
  
print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name='img'))
my_tag = {'name' : 'img', 'title' : 'Sunset Boulevard', 'src' : 'sunset.jpg', 'cls' : 'framed'}
print(tag(**my_tag)) # 딕셔너리 앞에 **을 붙이면 딕셔너리 안의 모든 항목이 인수로 전달됨

<br />
<p>hello<\p>
<p>hello<\p>
<p>world<\p>
<p class=sidebar>hello<\p>
<p class=sidebar>world<\p>
<img content=testing />
<img class=framed src=sunset.jpg title=Sunset Boulevard />


In [14]:
def f(a, *, b):
  return a , b

print(f(1, b=2)) 

(1, 2)


### 5.8. 매개변수에 대한 정보 읽기

In [0]:
import bobo

# Bobo는 hello()가 person 인수를 요구한다는 것을 알고 인수를 HTTP 요청에서 가져온다.
# 함수 객체 안의 __defaults__ 속성에 위치 인수와 키워드 인수의 기본값이 저장, 키워드 전용인수 값은 __kwdefaults__에

@bobo.query('/') # 데커레이터 : hello 함수와 프레임워크에서 제공하는 요청처리 메커니즘을 결합
def hello(person):
  return 'Hello %s!' % person

In [15]:
def clip(text, max_len=80):
  """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
  
  end = None
  if len(text) > max_len:
    space_before = text.rfind(' ', 0, max_len)
    if space_before >= 0:
      end = space_before
    else:
      space_after = text.rfind(' ', max_len)
      if space_after >=0 :
        end = space_after
  if end is None:
    end = len(text)
  return text[:end].rstrip()

print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames) # 지역변수도 포함
print(clip.__code__.co_argcount)
# 기본 변수 이름과 값, 변수 이름 등이 따로따로 저장되어 있어서 보기 불편함

(80,)
<code object clip at 0x7f04d0961270, file "<ipython-input-15-ec6457982db8>", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [16]:
from inspect import signature

sig = signature(clip)
print(sig)
print(str(sig))

for name, param in sig.parameters.items():
  print(param.kind, ":", name, "=", param.default)
  
  # POSITIONAL_OR_KEYWORD
  # VAR_POSITIONAL
  # VAR_KEYWORD
  # KEYWORD_ONLY
  # POSITIONAL_ONLY 로 매개변수 종류 구분

(text, max_len=80)
(text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [20]:
sig = signature(tag)
bound_args = sig.bind(**my_tag)
print(bound_args)

for name, value in bound_args.arguments.items():
  print(name, "=", value)
  
del my_tag['name']
bound_args = sig.bind(**my_tag) # 에러 발생 : missing a required argument: 'name'



<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


TypeError: ignored

### 5.9. 함수 애너테이션

함수의 매개변수와 반환값에 메타데이터를 추가할 수 있는 구문

In [21]:
# 각 매개변수 뒤에 :설명, 기본값 바로 앞에, 리턴값은 -> 으로
def clip(text:str, max_len:'int > 0'=80) -> str: # 함수 선언에 애너테이션 추가
  """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
  
  end = None
  if len(text) > max_len:
    space_before = text.rfind(' ', 0, max_len)
    if space_before >= 0:
      end = space_before
    else:
      space_after = text.rfind(' ', max_len)
      if space_after >=0 :
        end = space_after
  if end is None:
    end = len(text)
  return text[:end].rstrip()


clip.__annotations__

{'max_len': 'int > 0', 'return': str, 'text': str}

In [22]:
from inspect import signature

sig = signature(clip)
print(sig.return_annotation)

for param in sig.parameters.values():
  note = repr(param.annotation).ljust(13)
  print(note, ":", param.name, "=", param.default)



<class 'str'>
<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


### 5.10. 함수형 프로그래밍을 위한 패키지
#### 5.10.1 Operator 모듈



In [0]:
from functools import reduce

# 누적합의 경우에는 sum이 있지만
# 팩토리얼 같은 함수를 구현하기 위해서는 두 수의 곱을 행하는 람다 함수를 따로 정의해야함

def fact(n):
  return reduce(lambda a, b: a*b, range(1, n+1))

# 사소한 익명 함수를 작성하는 수고를 덜기 위해 operator 모듈은 수십개의 연산자에 대응하는 함수를 제공

from operator import mul
def fact(n):
  return reduce(mul, range(1, n+1))

In [26]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
for city in sorted(metro_areas, key=itemgetter(1)): # 1번 필드인 국가코드를 기준으로 정렬 lambda fields: fields[1]과 동일
  print(city)
  
cc_name = itemgetter(1, 0) # 해당 인덱스의 값들로 구성된 튜플을 반환
for city in metro_areas:
  print(cc_name(city))

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [31]:
from collections import namedtuple

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metro', 'name cc pop coord')

metro_data = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_areas]
print(metro_data[1])

print(metro_data[1].coord.lat)

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat') # 속성명에 .이 포함되어 있으면 내포된 객체를 찾아서 해당 속성을 가져온다

for city in sorted(metro_data, key=attrgetter('coord.lat')):
  print(name_lat(city))

Metro(name='Delhi NCR', cc='IN', pop=21.935, coord=LatLong(lat=28.613889, long=77.208889))
28.613889
('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [33]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

#iXXX는 복합 할당연산자

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [34]:
from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
hiphenate = methodcaller('replace', ' ', '-')
print(hiphenate(s))


THE TIME HAS COME
The-time-has-come


#### 5.10.2. functools.partial()로 인수 고정하기

partial()은 함수를 부분적으로 실행할 수 있게 해주는 고위 함수이다. 어떤 함수가 있을 때 partial()을 적용하면 원래 함수의 일부 인수를 고정한  콜러블을 생성한다.

In [36]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
print(triple(7))
print(list(map(triple, range(1,10))))

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [38]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(s1 == s2)
print(nfc(s1) == nfc(s2))

café café
False
True


In [42]:
picture = functools.partial(tag, 'img', cls='pic-frame') # tag 함수의 첫번째 인수로는 무조건 img, cls에는 pic-frame을 넘겨서 실행한다
print(picture(src='wumpus.jpeg'))
print(picture)
print(picture.func)
print(picture.args)
print(picture.keywords)

<img class=pic-frame src=wumpus.jpeg />
functools.partial(<function tag at 0x7f04d063c7b8>, 'img', cls='pic-frame')
<function tag at 0x7f04d063c7b8>
('img',)
{'cls': 'pic-frame'}


## 6. 일급 함수 디자인 패턴

언어의 종류에 따라 활용도 높은 디자인 패턴이 다르다. 일급함수를 지원하는 언어에서는 전략, 템플릿 메서드, 비지터 패턴 등을 활용하면 일부 클래스의 객체를 간단한 함수로 교체하여 반복되는 코드의 양을 상당히 줄일 수 있다.

### 6.1. 사례 : 전략 패턴의 리팩토링
> 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다. 

고객의 속성이나 주문한 상품에 따라서 다른 할인율을 적용하는 사례


In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount= 0
        else :
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
    
class Promotion(ABC): # 전략 : 추상 베이스 클래스
    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환한다."""
        
        
class FidelityPromo(Promotion): # 첫번째 구체적인 전략
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
    
class BulkItemPromo(Promotion): # 두번째 구체적인 전략
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
                
        return discount
    
    
class LargeOrderPromo(Promotion): # 세번째 구체적인 전략
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0
    
   

In [2]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)
cart = [LineItem("banana", 4, .5), LineItem("apple", 10, 1.5), LineItem("watermelon", 5, 5.0)]

print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))
print()

banana_cart = [LineItem("banana", 30, .5), LineItem("apple", 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo()))
print()

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>

<Order total: 30.00 due: 28.50>

<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


### 6.1.2. 함수지향 전략

위의 예제에서 구체적인 전략은 discount() 메서드 하나만 가진 클래스이다. 일반 함수와 큰 차이가 없기 때문에 리팩토링 여지가 있다.

In [6]:
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount= 0
        else :
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
       
def fidelity_promo(order): 
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
    
def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1

    return discount
    
    
def large_order_promo(order): 
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


In [7]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)
cart = [LineItem("banana", 4, .5), LineItem("apple", 10, 1.5), LineItem("watermelon", 5, 5.0)]

print(Order(joe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))
print()

banana_cart = [LineItem("banana", 30, .5), LineItem("apple", 10, 1.5)]
print(Order(joe, banana_cart, bulk_item_promo))
print()

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, large_order_promo))
print(Order(joe, cart, large_order_promo))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>

<Order total: 30.00 due: 28.50>

<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


### 6.1.3. 최선의 전략 선택하기 : 단순한 접근법

In [8]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


위의 예제는 간단하지만, 새로운 프로모션이 추가되는 경우 손이 많이 간다.

### 6.1.4. 모듈에서 전략 찾기


In [ ]:
# 모듈 전역에서 내부 조사해서 만든 promos 리스트
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

# promotions 모듈만 조사해서 
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

### 6.2. 명령

명령 패턴의 목적은 연산을 실행하는 객체와 연산을 구현하는 객체를 분리하는 것이다. 명령 객체를 수신자와 호출자 사이에 놓고, 명령은 execute() 메서드로 인터페이스를 구현한다. execute()는 원하는 수신자가 가지고 있는 메서드를 호출하도록 되어 있다. 

In [ ]:
class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    def __init__(self, commands):
        self.commands = list(commands)
        
    def __call__(self):
        for command in self.command:
            command()